In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import ARMAConv
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
from torch.utils.data import TensorDataset, DataLoader, Subset
import random
from torchvision import models

In [2]:
#data = np.load('pneumoniamnist_224.npz', allow_pickle=True)
data = np.load('pneumoniamnist_224.npz', allow_pickle=True)
# Combine train, val, and test sets
all_images = np.concatenate([data['train_images'], data['val_images'], data['test_images']], axis=0)
all_labels = np.concatenate([data['train_labels'], data['val_labels'], data['test_labels']], axis=0).squeeze()

In [3]:
images = all_images.astype(np.float32) / 255.0
images = np.repeat(images[:, None, :, :], 3, axis=1)  # Convert to 3 channels (N, 3, 224, 224)

# Convert to torch tensors
X = torch.tensor(images)
y = torch.tensor(all_labels).long()
print(X.shape, y.shape)

torch.Size([5856, 3, 224, 224]) torch.Size([5856])


In [4]:
dataset = TensorDataset(X, y)
class0_indices = [i for i in range(len(y)) if y[i] == 0]
class1_indices = [i for i in range(len(y)) if y[i] == 1]

random.seed(42)
sampled_class0 = random.sample(class0_indices, min(2000, len(class0_indices)))
sampled_class1 = random.sample(class1_indices, min(2000, len(class1_indices)))

combined_indices = sampled_class0 + sampled_class1
random.shuffle(combined_indices)

# Final subset
final_dataset = Subset(dataset, combined_indices)
final_loader = DataLoader(final_dataset, batch_size=64, shuffle=False)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Identity()  # Remove final classification layer
resnet = resnet.cuda() if torch.cuda.is_available() else resnet
resnet.eval()
resnet_feats = []
y_list = []

with torch.no_grad():
    for imgs, labels in final_loader:
        imgs = imgs.cuda() if torch.cuda.is_available() else imgs
        features = resnet(imgs)
        resnet_feats.append(features.cpu())
        y_list.extend(labels.cpu().tolist())
F = torch.cat(resnet_feats, dim=0).numpy().astype(np.float32)
y_labels = np.array(y_list).astype(np.float32)

print("Feature shape:", F.shape)
print("Label shape:", y_labels.shape)

/data/home/va797/tmp_pyg118/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/data/home/va797/tmp_pyg118/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Feature shape: (3583, 512)
Label shape: (3583,)


In [6]:
def create_adj(F, alpha=1):
    F_norm = F / np.linalg.norm(F, axis=1, keepdims=True)
    W = np.dot(F_norm, F_norm.T)
    W = (W >= alpha).astype(np.float32)
    return W

In [7]:
def asymmetrize_random(adj_matrix, seed=None):
    """
    Randomly orient each undirected edge from a symmetric adjacency matrix.
    """
    adj = np.array(adj_matrix, dtype=np.float32)
    n = adj.shape[0]
    asym = np.zeros((n, n), dtype=np.float32)
    rng = np.random.default_rng(seed)

    for i in range(n):
        for j in range(i + 1, n):
            if adj[i, j]:
                if rng.random() < 0.5:
                    asym[i, j] = adj[i, j]
                else:
                    asym[j, i] = adj[i, j]

    return asym

In [8]:
def load_data(adj, node_feats):
    node_feats = torch.from_numpy(node_feats).float()
    edge_index = torch.from_numpy(np.array(np.nonzero(adj))).long()
    return node_feats, edge_index

In [9]:
features = F # Use ResNet embeddings

device = 'cuda' if torch.cuda.is_available() else 'cpu'

W0 = create_adj(features, alpha=0.9)
# W_asym = asymmetrize_random(W0, seed=42)
node_feats, edge_index = load_data(W0, features)
data = Data(x=node_feats, edge_index=edge_index).to(device)
A = torch.from_numpy(W0).to(device)
print(data)

Data(x=[3583, 512], edge_index=[2, 1642013])


In [10]:
# features = F # Use ResNet embeddings

# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# W0 = create_adj(features, alpha=0.9)
# W_asym = asymmetrize_random(W0, seed=42)
# node_feats, edge_index = load_data(W_asym, features) # Use ResNet embeddings here as well
# data = Data(x=node_feats, edge_index=edge_index).to(device)
# A = torch.from_numpy(W_asym).to(device)
# print(data)

In [11]:
import torch.nn.functional as nnFn

class ARMAEncoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, device, activ="ELU", num_stacks=1, num_layers=1):
        super(ARMAEncoder, self).__init__()
        self.device = device
        # Define all activation functions
        activations = {
            "SELU": nnFn.selu,
            "SiLU": nnFn.silu,
            "GELU": nnFn.gelu,
            "ELU": nnFn.elu,
            "RELU": nnFn.relu
        }
        # Get the activation function based on the input string
        self.act = activations.get(activ, nnFn.elu)

        self.arma = ARMAConv(
            in_channels=input_dim,
            out_channels=hidden_dim,
            num_stacks=num_stacks,   # number of parallel stacks
            num_layers=num_layers,   # depth per stack
            act=self.act,               # nonlinearity inside ARMA
            shared_weights=True,     # weight sharing across layers
            dropout=0.25             # ARMA-internal dropout
        )
        self.batchnorm = nn.BatchNorm1d(hidden_dim)
        self.dropout = nn.Dropout(0.3)
        self.mlp = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.arma(x, edge_index)
        x = self.dropout(x)
        x = self.batchnorm(x)
        logits = self.mlp(x)
        return logits

In [12]:
class AvgReadout(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, seq, msk=None):
        if msk is None:
            return torch.mean(seq, 0)
        else:
            msk = torch.unsqueeze(msk, -1)
            return torch.sum(seq * msk, 0) / torch.sum(msk)

In [13]:
class Discriminator(nn.Module):
    def __init__(self, n_h):
        super().__init__()
        self.f_k = nn.Bilinear(n_h, n_h, 1)
        nn.init.xavier_uniform_(self.f_k.weight.data)
        if self.f_k.bias is not None:
            self.f_k.bias.data.fill_(0.0)

    def forward(self, c, h_pl, h_mi):
        c_x = torch.unsqueeze(c, 0).expand_as(h_pl)
        sc_1 = torch.squeeze(self.f_k(h_pl, c_x), 1)
        sc_2 = torch.squeeze(self.f_k(h_mi, c_x), 1)
        logits = torch.cat((sc_1, sc_2), 0)
        return logits

In [14]:
class DGI(nn.Module):
    def __init__(self, n_in, n_h, dropout=0.25):
        super().__init__()
        self.arma1 = ARMAEncoder(n_in, n_h, device='cuda' if torch.cuda.is_available() else 'cpu', activ=nn.ELU())
        self.read = AvgReadout()
        self.sigm = nn.Sigmoid()
        self.disc = Discriminator(n_h)

    def forward(self, seq1, seq2, edge_index):
        # Create Data objects for the ARMAEncoder
        data1 = Data(x=seq1, edge_index=edge_index)
        data2 = Data(x=seq2, edge_index=edge_index)

        h_1 = self.arma1(data1)
        c = self.read(h_1)
        c = self.sigm(c)
        h_2 = self.arma1(data2)
        logits = self.disc(c, h_1, h_2)
        return logits, h_1

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class DGI_with_classifier(DGI):
    def __init__(self, n_in, n_h, n_classes=2, cut=0, dropout=0.25):
        super().__init__(n_in, n_h, dropout=dropout)
        self.classifier = nn.Linear(n_h, n_classes)
        self.cut = cut

    def get_embeddings(self, node_feats, edge_index):
        _, embeddings = self.forward(node_feats, node_feats, edge_index)
        return embeddings

    def cut_loss(self, A, S):
        # ensure tensor
        if isinstance(A, np.ndarray):
            A = torch.from_numpy(A).float().to(S.device)

        S = F.softmax(S, dim=1)   # cluster assignment
        A_pool = torch.matmul(torch.matmul(A, S).t(), S)
        num = torch.trace(A_pool)

        D = torch.diag(torch.sum(A, dim=-1))
        D_pooled = torch.matmul(torch.matmul(D, S).t(), S)
        den = torch.trace(D_pooled)

        mincut_loss = -(num / den)

        St_S = torch.matmul(S.t(), S)
        I_S = torch.eye(S.shape[1], device=A.device)
        ortho_loss = torch.norm(St_S / torch.norm(St_S) - I_S / torch.norm(I_S))

        return mincut_loss + ortho_loss

    def modularity_loss(self, A, S):
        # ensure tensor
        if isinstance(A, np.ndarray):
            A = torch.from_numpy(A).float().to(S.device)

        C = F.softmax(S, dim=1)   # cluster assignment
        d = torch.sum(A, dim=1)
        m = torch.sum(A)

        B = A - torch.outer(d, d) / (2 * m)

        I_S = torch.eye(C.shape[1], device=A.device)
        k = torch.norm(I_S)
        n = S.shape[0]

        modularity_term = (-1 / (2 * m)) * torch.trace(torch.mm(torch.mm(C.t(), B), C))
        collapse_reg_term = (torch.sqrt(k) / n) * torch.norm(torch.sum(C, dim=0), p='fro') - 1

        return modularity_term + collapse_reg_term

    def Reg_loss(self, A, embeddings):
        # classifier output used as soft cluster assignment
        logits = self.classifier(embeddings)

        if self.cut == 1:
            return self.cut_loss(A, logits)
        else:
            return self.modularity_loss(A, logits)

In [16]:
hidden_dim = 256
cut = 0
dropout = 0.25
heads = 2
# make sure adjacency is a tensor on GPU
if isinstance(A, np.ndarray):
    A = torch.from_numpy(A).float().to(device)
else:
    A = A.float().to(device)

model = DGI_with_classifier(features.shape[1], hidden_dim, n_classes=2, cut=cut, dropout=dropout).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
bce_loss = nn.BCEWithLogitsLoss()

num_epochs = 5000
for epoch in range(num_epochs + 1):
    model.train()
    optimizer.zero_grad()

    perm = torch.randperm(node_feats.size(0))
    corrupt_features = node_feats[perm]

    logits, embeddings = model(node_feats.to(device), corrupt_features.to(device), edge_index.to(device))

    lbl = torch.cat([
        torch.ones(node_feats.size(0)),
        torch.zeros(node_feats.size(0))
    ]).to(device)

    dgi_loss = bce_loss(logits.squeeze(), lbl)
    reg_loss = model.Reg_loss(A, embeddings)
    loss = dgi_loss + 0.5 * reg_loss

    if epoch % 500 == 0:
        print(f"Epoch {epoch} | DGI Loss: {dgi_loss.item():.4f} | Reg Loss: {reg_loss.item():.4f} | Total: {loss.item():.4f}")

    loss.backward()
    optimizer.step()


Epoch 0 | DGI Loss: 0.7143 | Reg Loss: -0.2844 | Total: 0.5721
Epoch 500 | DGI Loss: 0.6932 | Reg Loss: -0.4670 | Total: 0.4596
Epoch 1000 | DGI Loss: 0.6987 | Reg Loss: -0.4655 | Total: 0.4660
Epoch 1500 | DGI Loss: 0.6932 | Reg Loss: -0.4676 | Total: 0.4593
Epoch 2000 | DGI Loss: 0.6931 | Reg Loss: -0.4679 | Total: 0.4592
Epoch 2500 | DGI Loss: 0.6933 | Reg Loss: -0.4671 | Total: 0.4597
Epoch 3000 | DGI Loss: 0.6933 | Reg Loss: -0.4673 | Total: 0.4596
Epoch 3500 | DGI Loss: 0.6932 | Reg Loss: -0.4679 | Total: 0.4592
Epoch 4000 | DGI Loss: 0.6932 | Reg Loss: -0.4679 | Total: 0.4592
Epoch 4500 | DGI Loss: 0.6931 | Reg Loss: -0.4680 | Total: 0.4591
Epoch 5000 | DGI Loss: 0.6932 | Reg Loss: -0.4679 | Total: 0.4593


In [19]:
model.eval()
with torch.no_grad():
    embeddings = model.get_embeddings(node_feats.to(device), edge_index.to(device))
    class_probabilities = F.softmax(model.classifier(embeddings), dim=1).cpu().numpy()

y_pred = np.argmax(class_probabilities, axis=1)

In [20]:
# Extract the true labels for the subset used for prediction
y_subset = y[combined_indices].cpu().numpy()

acc_score = accuracy_score(y_subset, y_pred)
acc_score_inverted = accuracy_score(y_subset, 1 - y_pred)
prec_score = precision_score(y_subset, y_pred)
rec_score = recall_score(y_subset, y_pred)
f1 = f1_score(y_subset, y_pred)
log_loss_value = log_loss(y_subset, class_probabilities)

print("Accuracy:", acc_score)
print("Accuracy (inverted):", acc_score_inverted)
print("Precision:", prec_score)
print("Recall:", rec_score)
print("F1:", f1)
print("Log Loss:", log_loss_value)

Accuracy: 0.08875244208763607
Accuracy (inverted): 0.9112475579123639
Precision: 0.13711990820424555
Recall: 0.1195
F1: 0.12770504942559444
Log Loss: 12.86645688544938


In [21]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss

hidden_dim   = 256
cut          = 0
dropout      = 0.25
num_runs     = 10
heads = 2
num_epochs   = 5000
lambda_list  = [0.001, 0.005, 0.009, 0.01, 0.05, 0.09, 0.1, 0.3, 0.5, 0.9, 1, 2, 5, 8]
base_seed    = 42

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

node_feats = node_feats.to(device)
edge_index = edge_index.to(device)
A = A.to(device)

# Use the y_subset created earlier
y_subset_np = y_subset.astype(int)

N, feats_dim = node_feats.size(0), node_feats.size(1)

all_results = []
bce_loss = nn.BCEWithLogitsLoss()

for lam in lambda_list:
    print(f"\n================ LAMBDA = {lam} ================\n")

    acc_scores, prec_scores, rec_scores, f1_scores, log_losses = [], [], [], [], []

    for run in range(num_runs):
        print(f"\n--- Run {run+1}/{num_runs} ---")

        seed = base_seed + run
        torch.manual_seed(seed)
        np.random.seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)


        model = DGI_with_classifier(features.shape[1], hidden_dim, n_classes=2, cut=cut, dropout=dropout).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.00001)
        scheduler = StepLR(optimizer, step_size=200, gamma=0.5)


        for epoch in range(num_epochs + 1):
            model.train()
            optimizer.zero_grad()

            perm = torch.randperm(N, device=device)
            corrupt_features = node_feats[perm]

            logits, embeddings = model(node_feats, corrupt_features, edge_index)

            lbl = torch.cat([torch.ones(N, device=device), torch.zeros(N, device=device)])
            dgi_loss = bce_loss(logits.squeeze(), lbl)
            reg_loss = model.Reg_loss(A, embeddings)

            loss = dgi_loss + lam * reg_loss

            if epoch % 500 == 0:
                print(f"Epoch {epoch:4d} | DGI: {dgi_loss.item():.4f} | Reg: {reg_loss.item():.4f} | "
                      f"λ*Reg: {(lam * reg_loss).item():.4f} | Total: {loss.item():.4f}")

            loss.backward()
            optimizer.step()
            scheduler.step()

        model.eval()
        with torch.no_grad():
            emb = model.get_embeddings(node_feats, edge_index)
            logits_cls = model.classifier(emb)                   # [N, 2]
            class_probabilities = F.softmax(logits_cls, dim=1).cpu().numpy()
            y_pred = np.argmax(class_probabilities, axis=1)

        acc  = accuracy_score(y_subset_np, y_pred)
        acc_inv = accuracy_score(y_subset_np, 1 - y_pred)

        if acc_inv > acc:
            acc = acc_inv
            y_pred = 1 - y_pred
            class_probabilities = class_probabilities[:, ::-1]

        prec = precision_score(y_subset_np, y_pred, zero_division=0)
        rec  = recall_score(y_subset_np, y_pred, zero_division=0)
        f1   = f1_score(y_subset_np, y_pred, zero_division=0)
        ll   = log_loss(y_subset_np, class_probabilities)

        print(f"Run {run+1} | Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1: {f1:.4f} | LogLoss: {ll:.4f}")

        acc_scores.append(acc)
        prec_scores.append(prec)
        rec_scores.append(rec)
        f1_scores.append(f1)
        log_losses.append(ll)

    lambda_results = {
        "lambda": lam,
        "accuracy":  (float(np.mean(acc_scores)), float(np.std(acc_scores))),
        "precision": (float(np.mean(prec_scores)), float(np.std(prec_scores))),
        "recall":    (float(np.mean(rec_scores)), float(np.std(rec_scores))),
        "f1":        (float(np.mean(f1_scores)),  float(np.std(f1_scores))),
        "log_loss":  (float(np.mean(log_losses)), float(np.std(log_losses))),
    }
    all_results.append(lambda_results)

    print(f"\n--- RESULTS FOR LAMBDA = {lam} ---")
    print(f"Accuracy : {lambda_results['accuracy'][0]:.4f} ± {lambda_results['accuracy'][1]:.4f}")
    print(f"Precision: {lambda_results['precision'][0]:.4f} ± {lambda_results['precision'][1]:.4f}")
    print(f"Recall   : {lambda_results['recall'][0]:.4f} ± {lambda_results['recall'][1]:.4f}")
    print(f"F1 Score : {lambda_results['f1'][0]:.4f} ± {lambda_results['f1'][1]:.4f}")
    print(f"Log Loss : {lambda_results['log_loss'][0]:.4f} ± {lambda_results['log_loss'][1]:.4f}")

print("\n================ FINAL SUMMARY FOR ALL LAMBDAS ================\n")
print(f"{'Lambda':>8} | {'Accuracy':>18} | {'Precision':>18} | {'Recall':>18} | {'F1 Score':>18} | {'Log Loss':>18}")
print("-" * 108)
for res in all_results:
    print(f"{res['lambda']:>8} | "
          f"{res['accuracy'][0]:.4f} ± {res['accuracy'][1]:.4f} | "
          f"{res['precision'][0]:.4f} ± {res['precision'][1]:.4f} | "
          f"{res['recall'][0]:.4f} ± {res['recall'][1]:.4f} | "
          f"{res['f1'][0]:.4f} ± {res['f1'][1]:.4f} | "
          f"{res['log_loss'][0]:.4f} ± {res['log_loss'][1]:.4f}")


================ LAMBDA = 0.001 ================


--- Run 1/10 ---
Epoch    0 | DGI: 0.7166 | Reg: -0.2839 | λ*Reg: -0.0003 | Total: 0.7164
Epoch  500 | DGI: 0.6931 | Reg: -0.4360 | λ*Reg: -0.0004 | Total: 0.6927
Epoch 1000 | DGI: 0.6931 | Reg: -0.4407 | λ*Reg: -0.0004 | Total: 0.6927
Epoch 1500 | DGI: 0.6931 | Reg: -0.4422 | λ*Reg: -0.0004 | Total: 0.6927
Epoch 2000 | DGI: 0.6931 | Reg: -0.4402 | λ*Reg: -0.0004 | Total: 0.6927
Epoch 2500 | DGI: 0.6931 | Reg: -0.4415 | λ*Reg: -0.0004 | Total: 0.6927
Epoch 3000 | DGI: 0.6931 | Reg: -0.4421 | λ*Reg: -0.0004 | Total: 0.6927
Epoch 3500 | DGI: 0.6931 | Reg: -0.4414 | λ*Reg: -0.0004 | Total: 0.6927
Epoch 4000 | DGI: 0.6931 | Reg: -0.4432 | λ*Reg: -0.0004 | Total: 0.6927
Epoch 4500 | DGI: 0.6931 | Reg: -0.4441 | λ*Reg: -0.0004 | Total: 0.6927
Epoch 5000 | DGI: 0.6931 | Reg: -0.4429 | λ*Reg: -0.0004 | Total: 0.6927
Run 1 | Accuracy: 0.9079 | Precision: 0.9563 | Recall: 0.8750 | F1: 0.9138 | LogLoss: 0.2658

--- Run 2/10 ---
Epoch    0 | DGI:

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from scipy.optimize import linear_sum_assignment

def cluster_acc(y_true, y_pred):
    """
    Calculate clustering accuracy. Assigns predicted clusters to true labels
    to maximize accuracy using the Jonker-Volgenant algorithm (linear_sum_assignment).
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    row_ind, col_ind = linear_sum_assignment(-w)
    return w[row_ind, col_ind].sum() / y_pred.size


model.eval()
with torch.no_grad():
    embeddings = model.get_embeddings(node_feats.to(device), edge_index.to(device))
    embeddings = embeddings.cpu().numpy()

kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
kmeans.fit(embeddings)
y_pred_kmeans = kmeans.labels_

ari_score = adjusted_rand_score(y, y_pred_kmeans)
nmi_score = normalized_mutual_info_score(y, y_pred_kmeans)

print("Adjusted Rand Score:", ari_score)
print("Normalized Mutual Information Score:", nmi_score)

acc_kmeans = cluster_acc(y, y_pred_kmeans)
print("Clustering Accuracy (mapped):", acc_kmeans)

In [ ]:
# class DGI(nn.Module):
#     def __init__(self, input_dim, hidden_dim,output_dim, cut=0):
#         super().__init__()
#         self.encoder = GCNEncoder(input_dim, hidden_dim)
#         self.readout = nn.Linear(hidden_dim, output_dim)
#         self.cut = cut
#         self.output_dim = output_dim

#     def forward(self, x, edge_index, corrupt_x, adj=None):
#         h = self.encoder(x, edge_index)
#         h_corrupt = self.encoder(corrupt_x, edge_index)

#         # Summary vector
#         s = torch.sigmoid(h.mean(dim=0))

#         # Positive & negative scores
#         pos = torch.matmul(h, s)
#         neg = torch.matmul(h_corrupt, s)

#         # DGI loss
#         dgi_loss = -torch.log(torch.sigmoid(pos - neg) + 1e-8).mean()

#         reg_loss = 0
#         if adj is not None:
#             A = torch.as_tensor(adj, dtype=torch.float32, device=x.device)
#             D = torch.diag(A.sum(dim=1))

#             if self.cut == 1:  # Cut loss
#                 L = D - A
#                 p = self.readout(h)
#                 C = F.softmax(p, dim=1)
#                 reg_loss = torch.trace(C.T @ L @ C) / (torch.trace(C.T @ D @ C) + 1e-8)

#             else:  # Modularity loss
#                 m = torch.sum(A)
#                 B = A - torch.outer(D.diag(), D.diag()) / (2 * m)
#                 p = self.readout(h)
#                 C = F.softmax(p, dim=1)
#                 k = torch.tensor(self.output_dim, dtype=torch.float32, device=x.device)
#                 n = C.shape[0]
#                 reg_loss = (-1 / (2 * m)) * torch.trace(torch.mm(torch.mm(C.t(), B), C))
#                 reg_loss += (torch.sqrt(k) / n) * torch.norm(torch.sum(C, dim=0), p='fro') - 1


#         return h, dgi_loss, reg_loss

In [ ]:
# hidden_dim = 256
# output_dim = 2
# cut = 0
# model = DGI(features.shape[1], hidden_dim, output_dim, cut=cut).to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# num_epochs = 7000
# for epoch in range(num_epochs+1):
#     model.train()
#     optimizer.zero_grad()

#     perm = torch.randperm(features.shape[0])
#     corrupt_features = node_feats[perm]

#     _, dgi_loss, reg_loss = model(
#         node_feats.to(device),
#         edge_index.to(device),
#         corrupt_features.to(device),
#         adj=torch.tensor(W0).to(device)
#     )

#     loss = dgi_loss + reg_loss
#     loss.backward()
#     optimizer.step()

#     if epoch % 500 == 0:
#         print(f"Epoch {epoch} | DGI Loss: {dgi_loss.item():.4f} | Reg Loss: {reg_loss.item():.4f} | Total Loss: {loss.item():.4f}")


In [ ]:
# model.eval()
# with torch.no_grad():
#     embeddings, _, _ = model(
#         node_feats.to(device),
#         edge_index.to(device),
#         node_feats.to(device),
#         adj=torch.tensor(W0).to(device)
#     )

# embeddings = embeddings.cpu().numpy()

In [ ]:
# from sklearn.cluster import KMeans
# from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

# # Use KMeans clustering
# kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
# kmeans.fit(embeddings)
# y_pred_kmeans = kmeans.labels_

# # Evaluate clustering performance
# ari_score = adjusted_rand_score(y, y_pred_kmeans)
# nmi_score = normalized_mutual_info_score(y, y_pred_kmeans)

# print("Adjusted Rand Score:", ari_score)
# print("Normalized Mutual Information Score:", nmi_score)

# # Note: K-Means is an unsupervised algorithm, so traditional classification metrics like accuracy, precision, recall, and F1 are not directly applicable without mapping clusters to classes.
# # However, we can calculate accuracy by mapping the cluster labels to the true labels in the way that maximizes accuracy.
# # This is not a standard evaluation for clustering but can give an idea of how well the clusters separate the classes.
# from scipy.optimize import linear_sum_assignment
# def cluster_acc(y_true, y_pred):
#     """
#     Calculate clustering accuracy. Assigns predicted clusters to true labels
#     to maximize accuracy using the Jonker-Volgenant algorithm (linear_sum_assignment).
#     """
#     y_true = y_true.astype(np.int64)
#     assert y_pred.size == y_true.size
#     D = max(y_pred.max(), y_true.max()) + 1
#     w = np.zeros((D, D), dtype=np.int64)
#     for i in range(y_pred.size):
#         w[y_pred[i], y_true[i]] += 1
#     row_ind, col_ind = linear_sum_assignment(-w)
#     return w[row_ind, col_ind].sum() / y_pred.size

# acc_kmeans = cluster_acc(y, y_pred_kmeans)
# print("Clustering Accuracy (mapped):", acc_kmeans)

1- GCN

Accuracy: 0.7466666666666667
Precision: 0.7263681592039801
Recall: 0.874251497005988
F1: 0.7934782608695652
Log Loss: 0.5786983582841999

Accuracy: 0.7533333333333333
Precision: 0.7360406091370558
Recall: 0.8682634730538922
F1: 0.7967032967032966
Log Loss: 0.5772490248961657